[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1oXrz0IL9rZwnwHijHWcXQcARkSyUPL_x#scrollTo=qTVVl7IyQ05H)


# dash-whales

**To run on Colab use the code below** :
```
git clone https://github.com/mohcineelharras/dash-whales
cd dash-whales
```

# Install necessary libraries
```
!pip install -r requirements.txt
!pip install dash
!pip install jupyter_dash
!pip install dash_bootstrap_components
!pip install ngrok
```


In [ ]:
!git clone https://github.com/mohcineelharras/dash-whales
!cd dash-whales


In [ ]:
!pip install -r requirements.txt
!pip install dash
!pip install jupyter_dash
!pip install dash_bootstrap_components
!pip install ngrok


## Scrap Data from etherscan
same code as in the file scrap_data.py

In [6]:
import requests
import time
import pandas as pd
import json
import os

# Etherscan API URL
url = "https://api.etherscan.io/api"

# Your Etherscan API Key
api_key = "VVJFE7IG5WSK8P2UJIJN5MST5UBQEZYVYW"

# Create output folder
if not os.path.exists("output"):
    os.makedirs("output")

# Load the JSON file into a dictionary
with open("ressources/dict_tokens_addr.json", "r") as file:
    dict_addresses = json.load(file)

def get_coin_data(tokenSymbol, contractAddr, n):

    # Get the latest block number
    params = {
        "module": "proxy",
        "action": "eth_blockNumber",
        "apikey": api_key
    }
    response = requests.get(url, params=params)
    latest_block = int(response.json()["result"], 16)

    # Initialize an empty DataFrame to store the transactions
    df_transactions = pd.DataFrame()

    # Define the number of transactions to retrieve per API call
    transactions_per_call = 10_000

    # Iterate 'n' times to retrieve 'n' sets of transactions
    for i in range(n):
        # Calculate the start and end blocks for each API call
        start_block = latest_block - (i + 1) * transactions_per_call
        end_block = latest_block - i * transactions_per_call

        # Make the API request
        params = {
            "module": "account",
            "action": "tokentx",
            "contractaddress": contractAddr,
            "startblock": start_block,
            "endblock": end_block,
            "sort": "asc",
            "apikey": api_key
        }
        response = requests.get(url, params=params)

        # Get the transactions data
        transactions = response.json()["result"]

        # Append the transactions to the DataFrame
        df_transactions = pd.concat([df_transactions, pd.DataFrame(transactions)])

        # Add a delay to avoid hitting the API rate limit
        time.sleep(1)

    df_transactions['timeStamp'] = pd.to_datetime(df_transactions['timeStamp'].astype(int), unit='s')
    df_transactions['value'] = df_transactions['value'].astype(float) / 1e18

    # Save transactions to a CSV file
    df_transactions.to_csv(f"output/transactions_{tokenSymbol}.csv", sep=";", index=False)
    print(f"All transactions saved to transactions_{tokenSymbol}_.csv")
    print()

# Define the number of blocks to retrieve transactions from
n_blocks = 20000
n_loop = n_blocks // 10_000


for tokenSymbol, contractAddr in dict_addresses.items():
    print(f"Key: {tokenSymbol}, Value: {contractAddr}")
    get_coin_data(tokenSymbol, contractAddr, n_loop)


Key: APE, Value: 0x4d224452801aced8b2f0aebe155379bb5d594381
All transactions saved to transactions_APE_.csv

Key: AXIE, Value: 0xbb0e17ef65f82ab018d8edd776e8dd940327b28b
All transactions saved to transactions_AXIE_.csv

Key: GALA, Value: 0xd1d2Eb1B1e90B638588728b4130137D262C87cae
All transactions saved to transactions_GALA_.csv

Key: MANA, Value: 0x0f5d2fb29fb7d3cfee444a200298f468908cc942
All transactions saved to transactions_MANA_.csv

Key: ILV, Value: 0x767fe9edc9e0df98e07454847909b5e959d7ca0e
All transactions saved to transactions_ILV_.csv



## WebApp on jupyter

In [5]:
import os
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Load the data from the CSV files
# Note: You need to replace 'output' with the actual path to your CSV files
dataframes = []
for filename in os.listdir('output'):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join('output', filename), sep=';')
        dataframes.append(df)
df = pd.concat(dataframes)

# Create the Dash app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the app layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1('Token Analysis'),
            dcc.Dropdown(
                id='token-dropdown',
                options=[{'label': i, 'value': i} for i in df['tokenSymbol'].unique()],
                value='MANA'
            ),
            # Add more filters here
        ], width=5),
        dbc.Col([
            dcc.Graph(id='token-graph')
        ], width=7)
    ])
])

# Define the callback to update the graph
@app.callback(
    Output('token-graph', 'figure'),
    [Input('token-dropdown', 'value')]
)
def update_graph(selected_token):
    filtered_df = df[df['tokenSymbol'] == selected_token]
    figure = go.Figure(
        data=[
            go.Scatter(
                x=filtered_df['timeStamp'],
                y=filtered_df['value'],
                mode='lines',
                name='Value over time'
            )
        ],
        layout=go.Layout(
            title='Token Value Over Time',
            yaxis=dict(
                title='Value ('+selected_token+')',  # Change this to 'Value (USD)' if the values are in USD
            ),
            showlegend=True,
            legend=go.layout.Legend(
                x=0,
                y=1.0
            ),
            margin=go.layout.Margin(l=40, r=0, t=40, b=30)
        )
    )
    return figure

# Run app and display result inline in the notebook
app.run_server(mode='inline')
